In [ ]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from mne.stats import permutation_cluster_test
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload
from scipy.stats import wilcoxon


# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization
import anal_functions

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

### Select Peaks and Plot Analytic Signal

In [ ]:
#Load raw fif data

test_raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'with_med_FTG',
        'Sub005_FIF.fif'
    )
)

subID = 'Sub005'
fft_name = str(subID) + '_'
print(fft_name)

In [ ]:
reload(dat_preproc)
x = test_raw.get_data() 
x1 = x[1,:]

peakMed = 77
peakStim = 65

dat_ngam = dat_preproc.low_highpass_filter(x1, peakMed-2, peakMed+2) 
dat_subh = dat_preproc.low_highpass_filter(x1, peakStim-2, peakStim+2) 
#dat_inb = dat_preproc.low_highpass_filter(x1, peakStim+3, peakMed-3) 

#datall = [dat_ngam, dat_subh, dat_inb] 
#labels = ['Peak'+str(peakMed)+'Hz','Peak'+str(peakStim)+'Hz', str(peakStim+3) + '-' + str(peakMed-3)+'Hz']

datall = [dat_ngam, dat_subh] 
labels = ['Peak'+str(peakStim)+'Hz']

print(labels)

In [ ]:
dat_subh.shape[0]

In [ ]:
%matplotlib qt
plt.plot(np.arange(0,dat_subh.shape[0]), dat_ngam)
plt.plot(np.arange(0,dat_subh.shape[0]), x[5,:])

In [ ]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [ ]:
%matplotlib qt
sm_signal_np = np.empty(shape = (1, x1.shape[0] - 499))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(1, 1, figsize=(12, 5))
wintosmooth = 500

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(np.squeeze(amplitude_envelope))

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    plt.plot(sm_signal, label = labels[idx])
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    #axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    #axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    plt.ylabel('Analytic Signal')
    plt.xlim([0, sm_signal.shape[0]])

    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 500 samples')
plt.legend(loc='upper right')

plt.show()


In [ ]:
x2 = x[4, :] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim)

In [ ]:
fig, ax1 = plt.subplots(figsize = (18,6))
#plt.rcParams['font.size'] = 10
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = labels[idx], lw = 2)
ax2.plot(sm_stim1[:], label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
#ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
#ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
#ax1.set_xlim(0, sm_signal_np.shape[0])
#ax1.set_xticks(np.arange(0, 100000, 20000))
#ax1.set_xticklabels(np.arange(0, 400, 80))
ax1.set_xlabel('Time [samples]')
plt.title(str(subID))
plt.show()

In [ ]:
#sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]], [sm_signal_np[1]],[sm_signal_np[2]],[sm_stim1]])))
sm_analSignal = np.transpose(np.squeeze(np.array([[sm_signal_np[0]],[sm_stim1]])))
sm_analSignal_df = pd.DataFrame(sm_analSignal, 
    columns = ['StimOn','StimVec'],
    )
print(sm_analSignal_df)

In [ ]:
fft_fig = os.path.join(project_path, 'figures','anal_signal','without_med_FTG/')
fft_file = os.path.join(project_path, 'data','anal_signal','without_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name)+'sm_analSignal',dpi = 300)
sm_analSignal_df.to_csv(str(fft_file)+str(fft_name)+'sm_analSignal.csv')

### Crop Analytic Signal to Epochs of Interest

Transform the analytic signal files to .fif objects

In [ ]:
##Convert analytic signal arrays to mne objects:
anal_signal_df = pd.read_csv(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'without_med_FTG',
    'Sub007_sm_analSignal.csv'
))

df_filtered = anal_signal_df.iloc[:, 1:]
dat_anal = df_filtered.values
ch_names = list(df_filtered.columns)
sfreq = 250
info = mne.create_info(ch_names, sfreq)
raw_anal = mne.io.RawArray(dat_anal.T, info)


In [ ]:
fif_name = os.path.join(project_path, 'data','anal_signal','without_med_FTG','Sub007'+'_AnalFIF.fif')
print(fif_name)

In [ ]:
raw_anal.save(fif_name)

In [ ]:
anal_epochs = pd.read_excel(os.path.join(
    project_path,
    'data', 'anal_signal',
    'Anal_epochs.xlsx'
))

anal_epochs
#subID = 'Sub005'
#fft_name = str(subID) + '_'
#print(fft_name)

In [ ]:
'''anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')'''

anal_fif = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'anal_signal',
        'with_med_FTG',
        'Sub065_AnalFIF.fif'
    )
)

subID = 'Sub005'
%matplotlib qt
anal_fif.plot(duration = 200)
'''cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 20) 

np.save(str(anal_file) + str(subID)+'sm_analSignal1TP.npy', cropped_anal_1tp)
plt.savefig(str(anal_fig) + str(subID)+'sm_analSignal2TP',dpi = 150)'''

In [ ]:
import glob
reload(anal_functions)
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')
anal_fig = os.path.join(project_path, 'figures','anal_signal', 'with_med_FTG/')


directory = os.path.join(project_path,
                         'data',
                         'anal_signal',
                         'with_med_FTG')  # Update with your directory path

# Create a file pattern to match .fif files
file_pattern = '*AnalFif_StimNorm.fif'

# Get a list of file paths that match the pattern
file_list = glob.glob(os.path.join(directory, file_pattern))

# Loop through the file list
for file_path in file_list:
    
    file_name = os.path.basename(file_path)
    subID = file_name[:6]

    anal_fif = mne.io.read_raw_fif(file_path)

    cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 30)

    np.save(str(anal_file) + str(subID)+'sm_analSignalSubh_On.npy', cropped_anal_1tp)
    plt.savefig(str(anal_fig) + str(subID)+'sm_analSignalSubh_On',dpi = 150)

    

In [ ]:
reload(anal_functions)
cropped_anal_2tp = anal_functions.anal_transitions_2tp(anal_epochs, anal_fif, subID, 20)


In [ ]:
anal_fig = os.path.join(project_path, 'figures','anal_signal', 'with_med_FTG/')
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')

plt.savefig(str(anal_fig) + str(subID)+'sm_analSignal2TP',dpi = 150)
np.save(str(anal_file) + str(subID)+'sm_analSignal2TP.npy', cropped_anal_2tp)

In [ ]:
reload(anal_functions)
cropped_anal_1tp = anal_functions.anal_transitions_1tp(anal_epochs, anal_fif, subID, 20)

In [ ]:
#plt.savefig(str(anal_fig) + str(subID)+'sm_analSignal1TP',dpi = 150)
anal_file = os.path.join(project_path, 'data','anal_signal', 'with_med_FTG/')
np.save(str(anal_file) + str(subID)+'sm_analSignal1TP.npy', cropped_anal_1tp)

### Average Cropped Analytic Signal and Plot

In [ ]:
5000/250

In [ ]:
%matplotlib qt
# Specify the directory path
directory = os.path.join(project_path, 'data', 'anal_signal', 'with_med_FTG')

# Find all .npy files in the directory ending with 'analSignal2TP.npy'
file_list = [file for file in os.listdir(directory) if file.endswith('Subh_On.npy')]

# Create an empty list to store the data arrays
data_list = []

# Loop through the file list and load the data
for file in file_list:
    data = np.load(os.path.join(directory, file))

    data_list.append(data)  # Select the first 10000 columns

# Stack the data arrays along the third axis
stacked_data = np.stack(data_list, axis=0)

# Calculate the mean and standard error over the 3rd dimension
mean_data = np.mean(stacked_data, axis=0)
sem_data = np.std(stacked_data, axis=0) / np.sqrt(stacked_data.shape[0])
xaxis = np.arange(1,15002)

plt.plot(xaxis, mean_data[0,:], label='Spontaneous', color = '#332288')
plt.fill_between(xaxis,mean_data[0,:] - sem_data[0,:], mean_data[0,:] + sem_data[0,:], color = '#332288', alpha=0.3)

plt.plot(xaxis, mean_data[1,:], label='Entrainment', color = '#CC6677', alpha = 0.3)
plt.fill_between(xaxis,mean_data[1,:] - sem_data[1,:], mean_data[1,:] + sem_data[1,:], color = '#CC6677', alpha = 0.2)

plt.plot(xaxis, mean_data[2,:], label='Intermediate', color = '#117733')
plt.fill_between(xaxis,mean_data[2,:] - sem_data[2,:], mean_data[2,:] + sem_data[2,:], color = '#117733', alpha=0.3)

plt.axvline(x=7500, color='grey', linestyle='--', lw = 3, alpha = 0.4, label = 'Entrainment On')


plt.ylim(0.6, 1.8)
plt.xticks(np.arange(0,17250,2500), labels=np.arange(-30,40,10))
plt.xlim(0, 10000)
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')
plt.legend()
plt.show()

In [ ]:
fft_fig = os.path.join(project_path, 'figures', 'anal_signal\\')

plt.savefig(str(fft_fig)+'AVG_AnalCropped_ENTRAINMENT_ON1',dpi = 300)
plt.savefig(str(fft_fig)+'AVG_AnalCropped_ENTRAINMENT_ON1.pdf', format='pdf')

### Make all Signals same Length

In [ ]:
anal_epochs = pd.read_excel(os.path.join(
    project_path,
    'data', 'anal_signal',
    'Anal_epochs.xlsx'
))

anal_epochs

In [ ]:
str(subID) + 'AnalFif_StimNorm.fif'

In [ ]:
import glob

directory = os.path.join(project_path,
                         'data',
                         'anal_signal',
                         'with_med_FTG//')  # Update with your directory path

# Create a file pattern to match .fif files
file_pattern = '*AnalFif_StimNorm.fif'

# Get a list of file paths that match the pattern
file_list = glob.glob(os.path.join(directory, file_pattern))

# Loop through the file list
for file_path in file_list:
    
    file_name = os.path.basename(file_path)
    subID = file_name[:6]

    anal_fif = mne.io.read_raw_fif(file_path, preload=True)

    filtered_df = anal_epochs[anal_epochs['Percept_ID'] == subID]
    
    rec_on = 1
    stim_on = filtered_df['Stim_On'].values[0]
    subh_on = filtered_df['Subh_On'].values[0]
    subh_off = filtered_df['Subh_Off'].values[0]

    part1 = anal_fif.copy().crop(tmin=rec_on, tmax = stim_on).get_data()
    part2 = anal_fif.copy().crop(tmin=stim_on, tmax = subh_on).get_data()
    part3 = anal_fif.copy().crop(tmin=subh_on, tmax = subh_off).get_data()
    #part4 = anal_fif.copy().crop(tmin=stim_off, tmax = rec_off).get_data()

    np.save(f'{directory}{subID}-Anal_Part1.npy', part1)
    np.save(f'{directory}{subID}-Anal_Part2.npy', part2)
    np.save(f'{directory}{subID}-Anal_Part3.npy', part3)
    #np.save(f'{directory}{subID}-Anal_Part4.npy', part4)

'''    #Normalize Stimulation Amplitude and express it between 0-100

    data_4th_channel = anal_fif.get_data()[3]

    # Normalize the values to range from 0 to 100, ignoring NaN values
    min_val = np.nanmin(data_4th_channel)
    max_val = np.nanmax(data_4th_channel)
    normalized_data = np.where(np.isnan(data_4th_channel), np.nan, ((data_4th_channel - min_val) / (max_val - min_val)) * 100)
    
    new_ch_name = 'StimNormalized'
    new_ch_data = [normalized_data]
    new_ch_unit = 'AU'  # Arbitrary unit, adjust as needed

    new_ch_info = mne.create_info(ch_names=[new_ch_name], sfreq=anal_fif.info['sfreq'], ch_types=['stim'])
    #new_ch_info['ch_names'] = [new_ch_name]

    # Create a new RawArray object with the normalized data and info
    new_ch_data = mne.io.RawArray(new_ch_data, new_ch_info)

    # Add the new channel to the existing data
    anal_fif.add_channels([new_ch_data])

    # Save the updated FiF file
    new_fif_file = str(subID) + 'AnalFif_StimNorm.fif'
    anal_fif.save(os.path.join(
        directory,
        new_fif_file), overwrite=True)'''


In [ ]:
plt.plot(arrays[0][0])
plt.show()

In [ ]:
### resample with scipy

# Specify the directory path
directory = os.path.join(
    project_path,
    'data', 'anal_signal', 'with_med_FTG'
)
NEW_N_SAMPLES = 5000

# Find all files in the directory ending with 'Anal_Part1.npy'
file_list = [file for file in os.listdir(directory) if file.endswith('Anal_Part3.npy')]

# Create an empty list to store the arrays
arrays = []

# Loop through the file list and load the arrays
for file in file_list:
    array = np.load(os.path.join(directory, file))
    arrays.append(array)


# Create a new list to store the resized arrays
resized_arrays = []

# Resize each array to the maximum length
for sub_arr in arrays:
    # resized_arr = np.resize(arr, (4, 50*250))
    resized_arr = np.zeros((sub_arr.shape[0], NEW_N_SAMPLES))
    for i_ch, ch in enumerate(sub_arr):
        # loop over bandwidths and stim-amps
        resized_ch = signal.resample(ch, num=NEW_N_SAMPLES)
        resized_arr[i_ch, :] = resized_ch

    resized_arrays.append(resized_arr)

# Print the resized arrays
#for i, arr in enumerate(resized_arrays):
#    print(f"Resized Array {i+1}:", arr)

In [ ]:
%matplotlib qt 

f, axes = plt.subplots(nrows=2, ncols=4)

for i, file in enumerate(file_list):
    array = np.load(os.path.join(directory, file))
    print(array.shape)
    ax = axes.flat[i]
    ax.plot(array[1,:])


In [ ]:
3000/250

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=4)

for i, arr in enumerate(resized_arrays):
    print(arr.shape)
    ax = axes.flat[i]
    ax.plot(arr[1,:])

In [ ]:
np.save(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'Part3_Resized.npy'),resized_arrays)

In [ ]:
part_list = np.load(os.path.join(
    project_path,
    'data',
    'anal_signal',
    'Part3_Resized.npy'))


#datall = [dat_ngam, dat_subh, dat_inb, stim] 

spontan_ch = part_list[:,0,:]
subh_ch = part_list[:,1,:]
inb_ch = part_list[:,2,:]
stim_ch = part_list[:,4,:]

mean_spontan = np.mean(spontan_ch, axis = 0)
sem_spontan = np.std(spontan_ch, axis=0) / np.sqrt(spontan_ch.shape[0])

mean_subh = np.mean(subh_ch, axis = 0)
sem_subh = np.std(subh_ch, axis=0) / np.sqrt(subh_ch.shape[0])

mean_inb = np.mean(inb_ch, axis = 0)
sem_inb = np.std(inb_ch, axis=0) / np.sqrt(inb_ch.shape[0])

mean_stim = np.mean(stim_ch, axis = 0)
sem_stim = np.std(stim_ch, axis=0) / np.sqrt(stim_ch.shape[0])

In [ ]:
xaxis = np.arange(0,part_list.shape[2])

plt.plot(xaxis,mean_spontan, label = 'Spontaneous FTG')
plt.fill_between(xaxis, mean_spontan - sem_spontan, mean_spontan + sem_spontan, alpha=0.3)

plt.plot(xaxis,mean_subh, label = 'Stim-induced FTG')
plt.fill_between(xaxis, mean_subh - sem_subh, mean_subh + sem_subh, alpha=0.3)

plt.plot(xaxis,mean_inb, label = 'Intermediate Activity')
plt.fill_between(xaxis, mean_inb - sem_inb, mean_inb + sem_inb, alpha=0.3)

plt.legend()

plt.show()

In [ ]:
import json
mean_vals = {
    'mean_spontan': mean_spontan,
    'sem_spontan': sem_spontan,
    'mean_subh': mean_subh,
    'sem_subh': sem_subh,
    'mean_inb': mean_inb,
    'sem_inb': sem_inb,
    'mean_stim': mean_stim,
    'sem_stim': sem_stim
}

# Specify the file path
file_path = os.path.join(
    project_path,
    'data',
    'anal_signal',
)

# Convert the dictionary to a DataFrame
mean_vals_df = pd.DataFrame.from_dict(mean_vals)

# Save the DataFrame as a CSV file
mean_vals_df.to_csv(os.path.join(file_path,'Part3_meanvals.csv'), index=False)


### Grand Plot with all activity over time

In [ ]:
my_path = os.path.join(
    project_path,
    'data',
    'anal_signal'
)

df_part1 = pd.read_csv(os.path.join(my_path, 'Part1_meanvals.csv'))
df_part2 = pd.read_csv(os.path.join(my_path, 'Part2_meanvals.csv'))
df_part3 = pd.read_csv(os.path.join(my_path, 'Part3_meanvals.csv'))

allparts_df = pd.concat([df_part1, df_part2, df_part3], axis = 0)

print(allparts_df.columns)

In [ ]:
allparts_df.iloc[:5000, allparts_df.columns.get_indexer(['mean_stim', 'sem_stim'])] = 0

In [ ]:
%matplotlib qt
fig, ax1 = plt.subplots(figsize = (18,6))

xaxis = np.arange(0,allparts_df.shape[0])

#stim_plot = allparts_df['mean_stim']/2 +0.5

stim_plot = allparts_df['mean_stim']
ax2 = ax1.twinx()

ax1.plot(xaxis,allparts_df['mean_spontan'], label = 'Spontaneous FTG', color = '#332288')
ax1.fill_between(xaxis, allparts_df['mean_spontan'] - allparts_df['sem_spontan'], allparts_df['mean_spontan'] + allparts_df['sem_spontan'], color = '#332288', alpha=0.3)

ax1.plot(xaxis,allparts_df['mean_subh'], label = 'Stim-induced FTG', color = '#CC6677')
ax1.fill_between(xaxis, allparts_df['mean_subh'] - allparts_df['sem_subh'], allparts_df['mean_subh'] + allparts_df['sem_subh'], color = '#CC6677', alpha=0.3)

ax2.plot(xaxis, stim_plot, label = 'Stimulation', alpha = 0.3)
ax2.fill_between(xaxis, stim_plot - allparts_df['sem_stim'], stim_plot + allparts_df['sem_stim'], alpha=0.2, color = 'grey')

ax1.plot(xaxis, allparts_df['mean_inb'], label = 'Intermediate Activity', color = '#117733')
ax1.fill_between(xaxis, allparts_df['mean_inb'] - allparts_df['sem_inb'], allparts_df['mean_inb'] + allparts_df['sem_inb'], color = '#117733', alpha=0.3)


plt.axvline(x=5000, color='gray', linestyle='--', linewidth=1.5, label = 'Stim On')
plt.axvline(x=15000, color='blue', linestyle='--', linewidth=1.5, label = 'Entrainment On')

ax1.set_ylim(0.5,2.5)


#ax2.set_yticks(np.arange(0.5,3,0.5), labels = np.arange(0,10,2))

plt.xlim(0,18800)
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Intensity [%]')
ax1.set_xlabel('Samples')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')

# Show the plot
plt.show()


In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'anal_signal',
    'TimeCourse_Gammas'
),dpi = 300)

plt.savefig(os.path.join(
    project_path,
    'figures',
    'anal_signal',
    'TimeCourse_Gammas.pdf'
))

### Statistics in averaged Anal Signal

In [ ]:
path_npy = '/Users//barbaramathiopoulou//Desktop//FTG_Figures//Anal_Signal/'

part1 = np.load(os.path.join(
    path_npy,'Part1_Resized.npy'
))

part2 = np.load(os.path.join(
    path_npy,'Part2_Resized.npy'
))

part3 = np.load(os.path.join(
    path_npy,'Part3_Resized.npy'
))

In [ ]:
part1_ftg = np.mean(part1[:,0,:],1)
part1_inter = np.mean(part1[:,2,:],1)
part1_subh = np.mean(part1[:,1,:],1)

part2_ftg = np.mean(part2[:,0,-5000:],1)
part2_inter = np.mean(part2[:,2,-5000:],1)
part2_subh = np.mean(part2[:,1,-5000:],1)

part3_ftg = np.mean(part3[:,0,:],1)
part3_inter = np.mean(part3[:,2,:],1)
part3_subh = np.mean(part3[:,1,:],1)

In [ ]:
part2_inter

In [ ]:
val1 = np.mean(part3[:,2,:],1)
val2 = np.mean(part3[:,1,:],1)

In [ ]:
statistic, p_value = wilcoxon(val1, val2)
print(np.round(p_value, decimals = 3))

In [ ]:
### PLOT BOXPLOTS
%matplotlib qt
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Part 1 subplot
axes[0].boxplot([part1_ftg, part1_inter, part1_subh], labels=['FTG', 'Intermediate', 'Entrainment'])
axes[0].plot([1, 2, 3], [part1_ftg, part1_inter, part1_subh], 'ko', markersize=8)
axes[0].plot([1, 2], [part1_ftg, part1_inter], 'k:', alpha=0.3)
axes[0].plot([2, 3], [part1_inter, part1_subh], 'k:', alpha=0.3)
axes[0].set_title('DBS Off')
axes[0].set_ylabel('Mean Analytic Signal')

# Part 2 subplot
axes[1].boxplot([part2_ftg, part2_inter, part2_subh], labels=['FTG', 'Intermediate', 'Entrainment'])
axes[1].plot([1, 2, 3], [part2_ftg, part2_inter, part2_subh], 'ko', markersize=8)
axes[1].plot([1, 2], [part2_ftg, part2_inter], 'k:', alpha=0.3)
axes[1].plot([2, 3], [part2_inter, part2_subh], 'k:', alpha=0.3)
axes[1].set_title('DBS On-Before Entrainment Onset')

# Part 3 subplot
axes[2].boxplot([part3_ftg, part3_inter, part3_subh], labels=['FTG', 'Intermediate', 'Entrainment'])
axes[2].plot([1, 2, 3], [part3_ftg, part3_inter, part3_subh], 'ko', markersize=8)
axes[2].plot([1, 2], [part3_ftg, part3_inter], 'k:', alpha=0.3)
axes[2].plot([2, 3], [part3_inter, part3_subh], 'k:', alpha=0.3)
axes[2].set_title('DBS On-After Entrainment Onset')


# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()



In [ ]:
plt.savefig('AnalSignalMeanBoxplots.pdf')
plt.savefig('AnalSignalMeanBoxplots', dpi=200)


### Switching Stim Off Analytic Signal for all

In [ ]:
stim_anal = []

ps_path = os.path.join(
   project_path,
   'data', 'anal_signal', 'switch_off'
)

for filename in os.listdir(ps_path):
   if filename.endswith('SWITCH_OFF.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_stim_anal = this_df['StimOn']

      stim_anal.append(this_stim_anal)



In [ ]:
all_stim_anal_df = pd.DataFrame(stim_anal)

mean_values = all_stim_anal_df.mean(skipna=True)
sem_values =  all_stim_anal_df.sem(skipna=True)


In [ ]:

# Plot the mean with shaded area for 
plot_mean = mean_values[:]
plot_sem = sem_values[:]
plt.plot(plot_mean, color='blue', label = 'Stim-Induced FTG')
plt.fill_between(plot_mean.index, plot_mean - plot_sem, plot_mean + plot_sem, color='lightblue')
plt.axvline(x = 5000, color = 'grey', ls='--', lw=3, alpha = 0.4, label = 'Stim Off')
plt.ylim(0.5,2)
plt.xlim(2500,7500)
plt.xticks(np.arange(2500,8000,500), labels=np.arange(-10,12,2))
plt.legend()
plt.xlabel('Time [sec]')
plt.ylabel('Z-scored Smoothed Analytic Signal')

In [ ]:
fft_fig = os.path.join(project_path, 'results\\')

plt.savefig(str(fft_fig)+'SWITCH_STIM_OFF',dpi = 250)